In [ ]:
from timeit import default_timer as time
import pandas as pd
import scipy.stats
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import calendar

import gc
from sklearn.ensemble import RandomForestClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb

%matplotlib inline
plt.rcParams.update({'figure.figsize':(12,6)})
plt.rcParams.update({'font.size':20})

# Modeling

In [ ]:
%%time
vars = pd.read_csv('https://raw.githubusercontent.com/Shawn-zou-2020/MyProjects/main/Fraud%20Analysis/Card%20Application%20Fraud%20Analysis/Raw%20Data/vars_final.csv')

In [ ]:
# use this to cap variables. For some problems it helps
Clip = 10

In [ ]:
vars=vars.rename(columns={'fraud_label':'Fraud'})

In [ ]:
vars['Fraud'].sum()

In [ ]:
vars.head(10)

In [ ]:
vars.shape

In [ ]:
vars.describe()

In [ ]:
record_save = vars['record']
Y_save = pd.DataFrame(vars.loc[:,'Fraud'])
Y_save.head()

In [ ]:
top30=['fulladdress_count_30',
 'ssn_lastname_count_30',
 'homephone_count_3',
 'ssn_count_14',
 'ssn_dob_count_0_by_30',
 'ssn_count_0_by_14',
 'fulladdress_homephone_count_7',
 'name_dob_count_14',
 'ssn_dob_count_7',
 'fulladdress_homephone_count_30',
 'ssn_count_0_by_30',
 'ssn_count_30',
 'ssn_count_7',
 'ssn_dob_count_14',
 'ssn_lastname_count_14',
 'ssn_name_count_0_by_14',
 'ssn_name_dob_count_0_by_14',
 'ssn_name_dob_count_14',
 'name_dob_count_0_by_14',
 'ssn_dob_count_30',
 'name_dob_count_7',
 'ssn_name_dob_count_0_by_30',
 'name_dob_count_0_by_30',
 'ssn_dob_count_0_by_14',
 'ssn_firstname_count_7',
 'ssn_name_dob_count_30',
 'ssn_firstname_count_14',
 'ssn_lastname_count_0_by_14',
 'ssn_firstname_count_0_by_14',
 'name_dob_count_30']
top30

In [ ]:
vars=vars.filter(top30[:25])
vars.shape

## Scale and truncate field values

In [ ]:
X_no_scaling = vars
X_no_scaling.describe()

In [ ]:
X = (X_no_scaling - X_no_scaling.mean()) / X_no_scaling.std()

In [ ]:
# push in any outlier values
cols = X.columns
X.loc[:,cols] = X[cols].clip(upper=Clip)
X.loc[:,cols] = X[cols].clip(lower=-1*Clip)
# X = (X_no_scaling - X_no_scaling.mean()) / X_no_scaling.std()
X.describe()

In [ ]:
# separate data into modeling (traintest) and out of time
X_trntst = X[0:833507]
Y_trntst = Y_save[0:833507]
X_oot = X[833507:]
Y_oot = Y_save[833507:]

In [ ]:
# %%time
# # Logistic regression

# m_i=30**(min(nitermax+1,6))

# for niter in range(nitermax):    
#     X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

#     model = LogisticRegression(max_iter=100, penalty='none', solver='saga')
 
#     X_oot = X_oot_orig.copy()
#     X_trn_save = X_trn.copy()
#     Y_trn_save = Y_trn.copy()

#     model.fit(X_trn, Y_trn.values.ravel())   

#     predictions = model.predict_proba(X_trn_save)[:,1]
#     X_trn['predicted'] = predictions
#     X_trn['Fraud'] = Y_trn_save['Fraud']
#     topRows = int(round(X_trn.shape[0]*0.03))
#     temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
#     needed = temp.loc[:,'Fraud']
#     FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

#     predictions = model.predict_proba(X_tst)[:,1]
#     X_tst['predicted']=predictions
#     X_tst['Fraud'] = Y_tst['Fraud']
#     topRows = int(round(X_tst.shape[0]*0.03))
#     temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
#     needed = temp.loc[:,'Fraud']
#     FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

#     predictions = model.predict_proba(X_oot)[:,1]
#     X_oot['predicted']=predictions
#     X_oot['Fraud'] = Y_oot['Fraud']
#     topRows = int(round(X_oot.shape[0]*0.03))
#     temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
#     needed = temp.loc[:,'Fraud']
#     FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
#     print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
# print(FDR3.mean())

## Models. You can comment out any of these cells and just explore one model type. You can also just rerun that single cell multiple times as you explore different model hyperparameters.

In [ ]:
# %%time
# # Single DT

# niter = 0
# nitermax = 5
# FDR3 = pd.DataFrame(np.zeros((nitermax,3)), columns=('trn', 'tst', 'oot'))
# X_oot_orig = X_oot.copy()
# pd.options.mode.chained_assignment = None  # default='warn'

# for niter in range(nitermax):    
#     X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

#     model = DecisionTreeClassifier(splitter='best',max_depth=30)
 
#     X_oot = X_oot_orig.copy()
#     X_trn_save = X_trn.copy()
#     Y_trn_save = Y_trn.copy()

#     model.fit(X_trn, Y_trn.values.ravel())   

#     predictions = model.predict_proba(X_trn_save)[:,1]
#     X_trn['predicted'] = predictions
#     X_trn['Fraud'] = Y_trn_save['Fraud']
#     topRows = int(round(X_trn.shape[0]*0.03))
#     temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
#     needed = temp.loc[:,'Fraud']
#     FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

#     predictions = model.predict_proba(X_tst)[:,1]
#     X_tst['predicted']=predictions
#     X_tst['Fraud'] = Y_tst['Fraud']
#     topRows = int(round(X_tst.shape[0]*0.03))
#     temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
#     needed = temp.loc[:,'Fraud']
#     FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

#     predictions = model.predict_proba(X_oot)[:,1]
#     X_oot['predicted']=predictions
#     X_oot['Fraud'] = Y_oot['Fraud']
#     topRows = int(round(X_oot.shape[0]*0.03))
#     temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
#     needed = temp.loc[:,'Fraud']
#     FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
#     print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
# print(FDR3.mean())

In [ ]:
# %%time
# # RF

# niter = 0
# nitermax = 1
# FDR3 = pd.DataFrame(np.zeros((nitermax,3)), columns=('trn', 'tst', 'oot'))
# X_oot_orig = X_oot.copy()
# pd.options.mode.chained_assignment = None  # default='warn'

# for niter in range(nitermax):    
#     X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

#     model = RandomForestClassifier(n_estimators=150, max_depth=30, min_samples_leaf=30, min_samples_split=500)
#     X_oot = X_oot_orig.copy()
#     X_trn_save = X_trn.copy()
#     Y_trn_save = Y_trn.copy()

#     model.fit(X_trn, Y_trn.values.ravel())   

#     predictions = model.predict_proba(X_trn_save)[:,1]
#     X_trn['predicted'] = predictions
#     X_trn['Fraud'] = Y_trn_save['Fraud']
#     topRows = int(round(X_trn.shape[0]*0.03))
#     temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
#     needed = temp.loc[:,'Fraud']
#     FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

#     predictions = model.predict_proba(X_tst)[:,1]
#     X_tst['predicted']=predictions
#     X_tst['Fraud'] = Y_tst['Fraud']
#     topRows = int(round(X_tst.shape[0]*0.03))
#     temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
#     needed = temp.loc[:,'Fraud']
#     FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

#     predictions = model.predict_proba(X_oot)[:,1]
#     X_oot['predicted']=predictions
#     X_oot['Fraud'] = Y_oot['Fraud']
#     topRows = int(round(X_oot.shape[0]*0.03))
#     temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
#     needed = temp.loc[:,'Fraud']
#     FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
#     print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
# print(FDR3.mean())

In [ ]:
# %%time
# # BT

# niter = 0
# nitermax = 5
# FDR3 = pd.DataFrame(np.zeros((nitermax,3)), columns=('trn', 'tst', 'oot'))
# X_oot_orig = X_oot.copy()
# pd.options.mode.chained_assignment = None  # default='warn'


# for niter in range(nitermax):    
#     X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)
    
#     model = lgb.LGBMClassifier(max_depth=5,learning_rate=0.01,n_estimators=500)

#     X_oot = X_oot_orig.copy()
#     X_trn_save = X_trn.copy()
#     Y_trn_save = Y_trn.copy()

#     model.fit(X_trn, Y_trn.values.ravel())   

#     predictions = model.predict_proba(X_trn_save)[:,1]
#     X_trn['predicted'] = predictions
#     X_trn['Fraud'] = Y_trn_save['Fraud']
#     topRows = int(round(X_trn.shape[0]*pp))
#     temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
#     needed = temp.loc[:,'Fraud']
#     FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

#     predictions = model.predict_proba(X_tst)[:,1]
#     X_tst['predicted']=predictions
#     X_tst['Fraud'] = Y_tst['Fraud']
#     topRows = int(round(X_tst.shape[0]*pp))
#     temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
#     needed = temp.loc[:,'Fraud']
#     FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

#     predictions = model.predict_proba(X_oot)[:,1]
#     X_oot['predicted']=predictions
#     X_oot['Fraud'] = Y_oot['Fraud']
#     topRows = int(round(X_oot.shape[0]*pp))
#     temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
#     needed = temp.loc[:,'Fraud']
#     FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
#     print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
# print(FDR3.mean())

In [ ]:
%%time
# BT

niter = 0
nitermax = 1
Fraud = pd.DataFrame(np.zeros((nitermax,3)), columns=('trn', 'tst', 'oot'))
X_oot_orig = X_oot.copy()
pd.options.mode.chained_assignment = None  # default='warn'

pp=[x * 0.01 for x in range(1, 6)]

for i in pp:
    for niter in range(nitermax):    
        X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

        model = lgb.LGBMClassifier(max_depth=3,learning_rate=0.1,n_estimators=300)

        X_oot = X_oot_orig.copy()
        X_trn_save = X_trn.copy()
        Y_trn_save = Y_trn.copy()

        model.fit(X_trn, Y_trn.values.ravel())   

        predictions = model.predict_proba(X_trn_save)[:,1]
        X_trn['predicted'] = predictions
        X_trn['Fraud'] = Y_trn_save['Fraud']
        topRows = int(round(X_trn.shape[0]*i))
        temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
        needed = temp.loc[:,'Fraud']
        Fraud.loc[niter, 'trn'] = sum(needed)

        predictions = model.predict_proba(X_tst)[:,1]
        X_tst['predicted']=predictions
        X_tst['Fraud'] = Y_tst['Fraud']
        topRows = int(round(X_tst.shape[0]*i))
        temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
        needed = temp.loc[:,'Fraud']
        Fraud.loc[niter, 'tst'] = sum(needed)

        predictions = model.predict_proba(X_oot)[:,1]
        X_oot['predicted']=predictions
        X_oot['Fraud'] = Y_oot['Fraud']
        topRows = int(round(X_oot.shape[0]*i))
        temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
        needed = temp.loc[:,'Fraud']
        Fraud.loc[niter, 'oot'] = sum(needed)
        
    print(i,Fraud.mean())

In [ ]:
# %%time
# # BT

# niter = 0
# nitermax = 5
# Fraud = pd.DataFrame(np.zeros((nitermax,3)), columns=('trn', 'tst', 'oot'))
# X_oot_orig = X_oot.copy()
# pd.options.mode.chained_assignment = None  # default='warn'

# pp=[x * 0.01 for x in range(1, 21)]

# for i in pp:
#     for niter in range(nitermax):    
#         X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

#         model = lgb.LGBMClassifier(max_depth=5,learning_rate=0.01,n_estimators=500)

#         X_oot = X_oot_orig.copy()
#         X_trn_save = X_trn.copy()
#         Y_trn_save = Y_trn.copy()

#         model.fit(X_trn, Y_trn.values.ravel())   

#         predictions = model.predict_proba(X_trn_save)[:,1]
#         X_trn['predicted'] = predictions
#         X_trn['Fraud'] = Y_trn_save['Fraud']
#         topRows = int(round(X_trn.shape[0]*i))
#         temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
#         needed = temp.loc[:,'Fraud']
#         Fraud.loc[niter, 'trn'] = sum(needed)

#         predictions = model.predict_proba(X_tst)[:,1]
#         X_tst['predicted']=predictions
#         X_tst['Fraud'] = Y_tst['Fraud']
#         topRows = int(round(X_tst.shape[0]*i))
#         temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
#         needed = temp.loc[:,'Fraud']
#         Fraud.loc[niter, 'tst'] = sum(needed)

#         predictions = model.predict_proba(X_oot)[:,1]
#         X_oot['predicted']=predictions
#         X_oot['Fraud'] = Y_oot['Fraud']
#         topRows = int(round(X_oot.shape[0]*i))
#         temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
#         needed = temp.loc[:,'Fraud']
#         Fraud.loc[niter, 'oot'] = sum(needed)
        
    
# print(i,Fraud.mean())

In [ ]:
# %%time
# # NN


# niter = 0
# nitermax = 6
# FDR3 = pd.DataFrame(np.zeros((nitermax,3)), columns=('trn', 'tst', 'oot'))
# X_oot_orig = X_oot.copy()
# pd.options.mode.chained_assignment = None  # default='warn'


# for niter in range(nitermax):    
#     X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

#     model = MLPClassifier(learning_rate='adaptive', hidden_layer_sizes=(20,20), activation='logistic')

#     X_oot = X_oot_orig.copy()
#     X_trn_save = X_trn.copy()
#     Y_trn_save = Y_trn.copy()

#     model.fit(X_trn, Y_trn.values.ravel())   

#     predictions = model.predict_proba(X_trn_save)[:,1]
#     X_trn['predicted'] = predictions
#     X_trn['Fraud'] = Y_trn_save['Fraud']
#     topRows = int(round(X_trn.shape[0]*0.03))
#     temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
#     needed = temp.loc[:,'Fraud']
#     FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

#     predictions = model.predict_proba(X_tst)[:,1]
#     X_tst['predicted']=predictions
#     X_tst['Fraud'] = Y_tst['Fraud']
#     topRows = int(round(X_tst.shape[0]*0.03))
#     temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
#     needed = temp.loc[:,'Fraud']
#     FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

#     predictions = model.predict_proba(X_oot)[:,1]
#     X_oot['predicted']=predictions
#     X_oot['Fraud'] = Y_oot['Fraud']
#     topRows = int(round(X_oot.shape[0]*0.03))
#     temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
#     needed = temp.loc[:,'Fraud']
#     FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
#     print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
# print(FDR3.mean())

In [ ]:
# %%time
# # ADB


# niter = 0
# nitermax = 5
# FDR3 = pd.DataFrame(np.zeros((nitermax,3)), columns=('trn', 'tst', 'oot'))
# X_oot_orig = X_oot.copy()
# pd.options.mode.chained_assignment = None  # default='warn'


# for niter in range(nitermax):    
#     X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

#     model = AdaBoostClassifier(n_estimators=200,learning_rate=1, algorithm='SAMME.R')

#     X_oot = X_oot_orig.copy()
#     X_trn_save = X_trn.copy()
#     Y_trn_save = Y_trn.copy()

#     model.fit(X_trn, Y_trn.values.ravel())   

#     predictions = model.predict_proba(X_trn_save)[:,1]
#     X_trn['predicted'] = predictions
#     X_trn['Fraud'] = Y_trn_save['Fraud']
#     topRows = int(round(X_trn.shape[0]*0.03))
#     temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
#     needed = temp.loc[:,'Fraud']
#     FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'Fraud'])

#     predictions = model.predict_proba(X_tst)[:,1]
#     X_tst['predicted']=predictions
#     X_tst['Fraud'] = Y_tst['Fraud']
#     topRows = int(round(X_tst.shape[0]*0.03))
#     temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
#     needed = temp.loc[:,'Fraud']
#     FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'Fraud'])

#     predictions = model.predict_proba(X_oot)[:,1]
#     X_oot['predicted']=predictions
#     X_oot['Fraud'] = Y_oot['Fraud']
#     topRows = int(round(X_oot.shape[0]*0.03))
#     temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
#     needed = temp.loc[:,'Fraud']
#     FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'Fraud'])
#     print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
# print(FDR3.mean())

# Hyperparameter Tuning

In [ ]:
%%time
# BT

niter = 0
nitermax = 1
Fraud = pd.DataFrame(np.zeros((nitermax,3)), columns=('trn', 'tst', 'oot'))
X_oot_orig = X_oot.copy()
pd.options.mode.chained_assignment = None  # default='warn'

pp=[x * 0.01 for x in range(1, 21)]



X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .25)

model = lgb.LGBMClassifier(max_depth=5,learning_rate=0.1,n_estimators=500)

X_oot = X_oot_orig.copy()
X_trn_save = X_trn.copy()
Y_trn_save = Y_trn.copy()

model.fit(X_trn, Y_trn.values.ravel())   

predictions = model.predict_proba(X_trn_save)[:,1]
X_trn['predicted'] = predictions
X_trn['Fraud'] = Y_trn_save['Fraud']


predictions = model.predict_proba(X_tst)[:,1]
X_tst['predicted']=predictions
X_tst['Fraud'] = Y_tst['Fraud']

predictions = model.predict_proba(X_oot)[:,1]
X_oot['predicted']=predictions
X_oot['Fraud'] = Y_oot['Fraud']
    
    
for i in pp:
    topRows = int(round(X_trn.shape[0]*i))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    Fraud.loc[i, 'trn'] = sum(needed)
    
    topRows = int(round(X_tst.shape[0]*i))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    Fraud.loc[i, 'tst'] = sum(needed)
    
    topRows = int(round(X_oot.shape[0]*i))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    Fraud.loc[i, 'oot'] = sum(needed)

Fraud

In [ ]:
sum(X_trn['Fraud'])

In [ ]:
sum(X_tst['Fraud'])

In [ ]:
sum(X_oot['Fraud'])